<a href="https://colab.research.google.com/github/jinnyjinny/DogFaceClassification/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Warning
- Set GPU mode (not to activate with CPU)

# Dependency

In [1]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 42kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=b179b75ab9510c3df43543df9e89a9988a14b10cf42d17184f687537e9dd20c6
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [2]:
import keras
import tensorflow as tf
import dlib, cv2, os 
from imutils import face_utils # imutils : 이미지  처리, dlib shape -> numpy ndarray로 자동변환
import numpy as np
import pandas  as pd
import matplotlib.pyplot as plt
import os
import csv

Using TensorFlow backend.


# Data Preprocessing

In [3]:
#### 삼각존 구하기 ### 

def landmark_ratio(DOG_FOLDER_PATH):

    ##### Load Model #####
    detector = dlib.cnn_face_detection_model_v1('/content/drive/My Drive/Cap/dogHeadDetector.dat')
    predictor = dlib.shape_predictor('/content/drive/My Drive/Cap/landmarkDetector.dat')

    ##### Load Image #####
    dog_folder_path = DOG_FOLDER_PATH
    dog_folder_list = os.listdir(dog_folder_path)
    dog_folder_list.sort()
    
    landmarks = {}
    for f in dog_folder_list:
        img_path = dog_folder_path + '/' + f
        img = cv2.imread(img_path)
        index = str(f.split('.')[0]) 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        ##### Detect Face #####
        dets = detector(img, upsample_num_times=1)
        img_result = img.copy()

        for i, d in enumerate(dets):
            #print("Detection {}: Left: {} Top: {} Right: {} Bottom: {} Confidence: {}".format(i, d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom(), d.confidence))
            x1, y1 = d.rect.left(), d.rect.top()
            x2, y2 = d.rect.right(), d.rect.bottom()
            cv2.rectangle(img_result, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=(255,0,0), lineType=cv2.LINE_AA)
        
        
        ##### Detect Landmarks #####
        for i, d in enumerate(dets):
            shape = predictor(img, d.rect) # detect in range of d.rect 
            shape = face_utils.shape_to_np(shape)
            shape = shape.reshape(-1) # x0 y0 x1 y1 x2 y2 x3 y3 x4 y4 x5 y5
            ##### Landmarks Ratio #####
            x3 = int(shape[6])
            y3 = int(shape[7])
            x5 = int(shape[10])
            y5 = int(shape[11])
            x2 = int(shape[4])
            y2 = int(shape[5])
            area = abs ( (x5 - x3) * (y2 - y3) - (y5 - y3) * (x2 - x3) )
            AB = ( (x5 - x2) ** 2 + (y5 - y2) ** 2 ) ** 0.5
            tri_zone =  (area / AB) * 0.01
            tri_zone = [round(tri_zone, 4)]

            ##### Save CSV #####
            id = str(landmarks.keys())
            df = pd.DataFrame(tri_zone, index = ['tri_zone'])
            df = df.T
         
            with open(TOTAL_FOLDER_PATH + '/train_landmark_ratio_0729.csv','a',newline='') as file :
                writer = csv.writer(file)
                writer.writerow(tri_zone)

# Main

In [4]:
DOG_FOLDER_PATH = '/content/drive/My Drive/Cap/data/real/train/zzo'
TOTAL_FOLDER_PATH = '/content/drive/My Drive/Cap/data/real'

with open(TOTAL_FOLDER_PATH + '/train_landmark_ratio_0729.csv','w',newline='') as file :
      writer = csv.writer(file)
      writer.writerow(['tri_zone'])

In [5]:
zzo_folder_landmark = landmark_ratio(DOG_FOLDER_PATH)